**NOTE: This notebook is written for the Google Colab platform. However it can also be run (possibly with minor modifications) as a standard Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, KBinsDiscretizer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from class_utils import error_histogram
import matplotlib.pyplot as plt

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
DATA_HOME = "https://github.com/michalgregor/ml_notebooks/blob/main/data/{}?raw=1"

from class_utils.download import download_file_maybe_extract
download_file_maybe_extract(DATA_HOME.format("sigmoid_regression_data.csv"), directory="data")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Regression and Corresponding Data Preprocessing

As our next example we will show how to solve a simple regression task. As our regression method (just as in the case of classification) we will use the $k$ nearest neighbours method (KNN). For classification the prediction was determined by voting among the $k$ nearest neighbours. In the case of regression the prediction is formed by computing the average of the $k$ nearest neighbours' values. 

### Loading the Dataset

Our example will begin as usual: by loading a dataset from a CSV file: in this case the file will only have two columns. The first column, $x$, will represent the input of our model and the second, $y$, will represent the desired output.



In [ ]:
df = pd.read_csv("data/sigmoid_regression_data.csv")
df.head()

### Stratification When Splitting the Dataset

The next step, of course, is to split the dataset into the train and the test set. Let us recall that we used stratification by the label when splitting the dataset for a classification task. This was to ensure that the ratio of the classes in the training and the testing set will remain close to that in the original dataset.

Obviously in the case of regression we cannot do the same thing – or at least not quite as easily as for classification. This is because the desired outputs are now continuous. In some cases, however, it may still be desirable to apply some kind of stratification – especially if the dataset is small, as it is in our case. There is otherwise a risk that the training and the testing set will not be representative.

To show how such problem might manifest itself is very easy: all we need to do is to split our dataset in the normal way without using stratification and to visualize the training and the testing set in the same plot:



In [ ]:
df_train, df_test = train_test_split(df, test_size=0.3, random_state=4)

In [ ]:
plt.scatter(df_train['x'], df_train['y'], marker='x', label="tréningové dáta")
plt.scatter(df_test['x'], df_test['y'], c='r', label="testovacie dáta")
plt.xlabel('x')
plt.ylabel('y')
plt.grid(ls='--')
plt.legend()
plt.savefig("output/regression_split_plain.pdf", bbox_inches='tight', pad_inches=0)

What we should be able to see in the plot is that the testing data does not even begin to cover the entire space. So how do we apply stratification to prevent this problem? Given that the column by which we stratify should be discrete, the simplest solution is, of course, to discretize our continuous column – e.g. using the `KBinsDiscretizer` transformer from `scikit-learn`. The discretization itself might look as follows:



In [ ]:
kbins = KBinsDiscretizer(6, encode='ordinal')
y_stratify = kbins.fit_transform(df[['y']])

The discretized column `y_stratify` can now be used for stratification:



In [ ]:
df_train, df_test = train_test_split(df, stratify=y_stratify,
                                 test_size=0.3, random_state=4)

When we visualize the results of such splitting, we should be able to observe that the samples have been split into the training and the testing set much more uniformly:



In [ ]:
plt.scatter(df_train['x'], df_train['y'], marker='x', label="training data")
plt.scatter(df_test['x'], df_test['y'], c='r', label="testing data")
plt.xlabel('x')
plt.ylabel('y')
plt.grid(ls='--')
plt.legend()
plt.savefig("output/regression_split_stratif.pdf", bbox_inches='tight', pad_inches=0)

---
### Task 1: Constructing the Pipeline

**The next step is, as usual, to construct the preprocessing pipeline. As the first step, determine which input columns are categorical and which are numeric and then create the same pipeline object that we have been using the previous examples.** 

NOTE: It will not be necessary to preprocess the output column in our case.

---


In [ ]:
categorical_inputs = [           ]  # ----

numeric_inputs = [               ]  # ----

output = 'y'

In [ ]:
input_preproc = make_column_transformer(
    
    
    
    # ----


    

### Data Preprocessing

The data will be preprocessed in the usual way:



In [ ]:
X_train = input_preproc.fit_transform(df_train[categorical_inputs+numeric_inputs])
Y_train = df_train[output].values

X_test = input_preproc.transform(df_test[categorical_inputs+numeric_inputs])
Y_test = df_test[output].values

### Training the Model

Our model will be trained using the unified interface – we will only need to change its type. We will be using a `KNeighborsRegressor` instead of a `KNeighborsClassifier`.



In [ ]:
model = KNeighborsRegressor(n_neighbors=5)
model.fit(X_train, Y_train)

### Testing the Model

Finally the behaviour of our model will be evaluated on the testing data. For regression we will naturally not use accuracy as a performance metric – regression has its own indicators. Among the most frequently used ones are the **mean squared error**  (MSE):
\begin{equation}
MSE = \frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y}_i)^2.
\end{equation}
where $y_i$ is the desired output and $\hat{y}_i$ is the actual output of our model for sample $i$ from the dataset and $N$ is the total number of samples in the dataset.

and the **mean absolute error**  (MAE):
\begin{equation}
MAE = \frac{1}{N} \sum_{i=1}^{N} |y_i - \hat{y}_i|.
\end{equation}

Mean squared error is more sensitive to large errors (due to the square). The advantage of mean absolute error is that it from the same scale as the data and it is therefore easier to interpret (even though it is alway possible to take the square root of the mean squared error as well). There is, of course, a number of other indicators (e.g. percentage errors, the coefficient of determination, etc.), but we will not be considering these in the present notebook.

Let us then use our regression model and compute predictions for the testing set.



In [ ]:
y_test = model.predict(X_test)

We will compute and display the mean square and absolute error. Let us recall that the range of the outputs is approximately 0 to 1.



In [ ]:
mse = mean_squared_error(Y_test, y_test)
mae = mean_absolute_error(Y_test, y_test)

print("MSE: {}".format(mse))
print("MAE: {}".format(mae))

To get a fuller idea of how the errors are distributed we can also visualize the entire distribution using the output and error histogram. We will standardize the outputs for the purposes of this plot to get the histograms of the outputs and the error to align. We will also use scaled data to compute the mean absolute error to ensure that it is on the same scale.



In [ ]:
plt.figure(figsize=(8, 6))
error_histogram(Y_test, y_test, Y_fit_scaling=Y_train)
plt.savefig("output/error_output_histogram.pdf", bbox_inches='tight', pad_inches=0)

### Visualizing the Regression Relationship

Given that our regression relationship is 2-dimensional, we can easily visualize it in a 2D plot and evaluate it visually as well.



In [ ]:
x_min = min(np.min(X_train), np.min(X_test))
x_max = max(np.max(X_train), np.max(X_test))

xx = np.linspace(x_min, x_max, 250).reshape((-1, 1))
yy = model.predict(xx)

plt.scatter(X_train, Y_train, marker='x', label="training data")
plt.scatter(X_test, Y_test, c='r', label="testing data")

plt.plot(xx, yy, label="regression curve")

plt.xlabel('x')
plt.ylabel('y')
plt.grid(ls='--')
plt.legend()

plt.savefig("output/knn_regression.pdf", bbox_inches="tight", pad_inches=0)

One thing that we should notice in the plot is that the regression relationship is not smooth. This is, of course, because there are areas, where all the points share the same nearest neighbours and the output is therefore insensitive to the input unless it leaves the area. In later notebooks we will explore other kinds of models, which will be able to achieve better results.

